In [1]:
from textProcessing import *
from evaluation import *
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter
import numpy as np
import main
import pandas as pd
import pickle
from num2words import num2words
from paths import *

In [2]:
tfidf_matrix = pickle.load(open(paths.dataset_two_tfidf_matrix, "rb"))
vectorizer = pickle.load(open(paths.dataset_two_model, "rb"))
qr_path = paths.dataset_two_question_forum
processed_queries = textProcessing.ds_pre_processing2(qr_path)
cleanedQueries1=[]
for doc in processed_queries:
    doc=' '.join(doc)
    cleanedQueries1.append(doc)

In [3]:
mappings = evaluation.golden_standard(paths.dataset_two_qas_forum)

In [4]:
print(mappings)

{'0': [130975, 130977, 130982, 130991, 130997, 130998, 131019, 131021, 131026, 135480, 139194, 143870, 145486, 182816], '1': [152301, 152331, 152343, 152442, 169727], '2': [109883, 109890, 111423, 111459, 123072, 123190, 123866, 126082, 130382, 136722, 143233, 143357, 147092, 164706, 176593], '3': [110334, 110335, 110339, 110340, 110342, 110607, 112222, 116566, 117010, 118889, 119859, 120072, 121325, 127544, 127555, 133235, 133554, 134605, 135862, 140159, 142895, 156516, 157720, 167062, 174912, 181003, 182596, 187012, 191960, 192588, 207111], '4': [153672], '5': [110194, 110197, 110199, 110204, 110219, 111192, 112001, 113065, 116405, 124755, 125721, 127353, 131043, 134664, 138370, 155507, 163753, 170348, 179085], '6': [112245, 112248, 114565, 159985], '7': [109846, 109848, 109849, 109882, 110075, 110504, 111812, 112282, 115381, 118513, 128580, 137577, 156965, 157271, 171787], '8': [157225, 157226, 157227, 157239, 157240, 157248, 157349, 157360, 157461, 157814, 189538], '9': [150094, 15

In [5]:
precision_all = 0.0
recall_all = 0.0
mean_avg_precision = 0.0
mean_reciprocal_rank = 0.0
rr = 0.0 
for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    model_output = []
    query_vector = vectorizer.transform([cleanedQueries1[int(query_id)]])
    cos=cosine_similarity(tfidf_matrix,query_vector).flatten()
    map=dict()
    for x in range(cos.size):
        map[cos[x]]=x

    myKey = list(map.keys())
    myKey.sort(reverse=True)
    if len(myKey) >= 10:
        for i in range(10):
            model_output.append(map.get(myKey[i]))
    else:
        # Handle the case where myKey has fewer than 10 elements
        for key in myKey:
            model_output.append(map.get(key))
        
    
    avg_precision = evaluation.calculate_average_precision(model_output,gold_standard)
    print("avg_precision(" + str(query_id) + "): " + str(avg_precision))
    precision_all += avg_precision
    
    recall = evaluation.calculate_recall_at_k(model_output, gold_standard)
    print("recall(" + str(query_id) + "): " + str(recall))
    
    recall_all += recall
    
    reciprocal_rank = evaluation.calculate_reciprocal_rank(model_output, gold_standard)
    print("reciprocal_rank(" + str(query_id) + "): " + str(reciprocal_rank))
    print("-------------------------------")
    
    rr += reciprocal_rank
avg_recall = recall_all / float(len(mappings.keys()))
mean_avg_precision = precision_all / float(len(mappings.keys()))
mean_reciprocal_rank = rr / float(len(mappings.keys()))

print("-------------------------------")

print("mean average precision = " + str(mean_avg_precision))
print("mean reciprocal rank = " + str(mean_reciprocal_rank))
print("average recall = " + str(avg_recall))

avg_precision(0): 0.24285714285714285
recall(0): 0.14285714285714285
reciprocal_rank(0): 0.2
-------------------------------
avg_precision(1): 0.16666666666666666
recall(1): 0.2
reciprocal_rank(1): 0.16666666666666666
-------------------------------
avg_precision(2): 0.4444444444444444
recall(2): 0.2
reciprocal_rank(2): 0.5
-------------------------------
avg_precision(3): 0.1
recall(3): 0.03225806451612903
reciprocal_rank(3): 0.1
-------------------------------
avg_precision(4): 0
recall(4): 0.0
reciprocal_rank(4): 0
-------------------------------
avg_precision(5): 0.1111111111111111
recall(5): 0.05263157894736842
reciprocal_rank(5): 0.1111111111111111
-------------------------------
avg_precision(6): 0.3666666666666667
recall(6): 0.5
reciprocal_rank(6): 0.3333333333333333
-------------------------------
avg_precision(7): 0
recall(7): 0.0
reciprocal_rank(7): 0
-------------------------------
avg_precision(8): 0.5404761904761904
recall(8): 0.36363636363636365
reciprocal_rank(8): 1.0
-

_____________________
AFTER CLUSTERING 
________________________________________

In [8]:
kmeans = pickle.load(open(paths.clustered_dataset_two, "rb"))
vectorizer = pickle.load(open(paths.dataset_two_model, "rb"))
tfidf_matrix =  pickle.load(open(paths.dataset_two_tfidf_matrix, "rb"))
qr_path = paths.dataset_two_question_forum

In [9]:
processed_queies = textProcessing.ds_pre_processing2(qr_path)
cleanedQueries1=[]
for doc in processed_queies:
    doc=' '.join(doc)
    cleanedQueries1.append(doc)

In [10]:
mappings = evaluation.golden_standard(paths.dataset_two_qas_forum)

In [11]:
df_matrix = pd.DataFrame(tfidf_matrix)

cluster_map = pd.DataFrame()
cluster_map['doc_id'] = df_matrix.index.values
cluster_map['doc_vectors'] = df_matrix
cluster_map['cluster'] = kmeans.labels_

def get_vectors_of_one_cluster(query_cluster):
        
        one_cluster = cluster_map[cluster_map.cluster == query_cluster]
        
        return one_cluster

In [12]:
def cos_simi(cluster_vectors,query_vector,one_cluster_with_id):
        cos=cosine_similarity(cluster_vectors,query_vector).flatten()

        model_output = []
        map=dict()
        for x in range(cos.size):
            y=one_cluster_with_id['doc_id'].iloc[x]
            map[cos[x]]=y

        myKey = list(map.keys())
        myKey.sort(reverse=True)
        if len(myKey) >= 10:
            for i in range(10):
                model_output.append(map.get(myKey[i]))
        else:
            # Handle the case where myKey has fewer than 10 elements
            for key in myKey:
                model_output.append(map.get(key))
            
        return model_output


In [13]:
precision_all = 0.0
recall_all = 0.0
mean_avg_precision = 0.0
mean_reciprocal_rank = 0.0
rr=0.0
for query_id in mappings.keys():
    gold_standard = mappings.get(str(query_id))
    
    query_vector = vectorizer.transform([cleanedQueries1[int(query_id)]])
    #get query cluster
    query_cluster = kmeans.predict(np.array([query_vector.toarray().flatten()]))
    #get cluster vectors
    cluster_labels = kmeans.labels_  
    cluster_indices = np.where(cluster_labels == query_cluster)[0]
    cluster_vectors = tfidf_matrix[cluster_indices]
    #get document vectors of the same cluster as query
    one_cluster_with_id = get_vectors_of_one_cluster(query_cluster[0])
    #cosine similarity
    model_output = cos_simi(cluster_vectors,query_vector,one_cluster_with_id)


    avg_precision =evaluation.calculate_average_precision(model_output,gold_standard)
    print("avg_precision(" + str(query_id) + "): " + str(avg_precision))
    precision_all += avg_precision
    
    recall = evaluation.calculate_recall_at_k(model_output, gold_standard)
    print("recall(" + str(query_id) + "): " + str(recall))
    recall_all+=recall
    reciprocal_rank = evaluation.calculate_reciprocal_rank(model_output, gold_standard)
    print("reciprocal_rank(" + str(query_id) + "): " + str(reciprocal_rank))
    
    rr += reciprocal_rank
avg_recall = recall_all / float(len(mappings.keys()))
mean_avg_precision = precision_all / float(len(mappings.keys()))
mean_reciprocal_rank = rr / float(len(mappings.keys()))

print("-------------------------------")
print("mean average precision = " + str(mean_avg_precision))
print("mean reciprocal rank = " + str(mean_reciprocal_rank))   
print("average recall = " + str(avg_recall))

avg_precision(0): 0.24285714285714285
recall(0): 0.14285714285714285
reciprocal_rank(0): 0.2
avg_precision(1): 0.25
recall(1): 0.2
reciprocal_rank(1): 0.25
avg_precision(2): 0.5166666666666666
recall(2): 0.26666666666666666
reciprocal_rank(2): 0.5
avg_precision(3): 0.22619047619047616
recall(3): 0.06451612903225806
reciprocal_rank(3): 0.16666666666666666
avg_precision(4): 0
recall(4): 0.0
reciprocal_rank(4): 0
avg_precision(5): 0.1111111111111111
recall(5): 0.05263157894736842
reciprocal_rank(5): 0.1111111111111111
avg_precision(6): 0.26666666666666666
recall(6): 0.5
reciprocal_rank(6): 0.3333333333333333
avg_precision(7): 0
recall(7): 0.0
reciprocal_rank(7): 0
avg_precision(8): 1.0
recall(8): 0.09090909090909091
reciprocal_rank(8): 1.0
avg_precision(9): 0
recall(9): 0.0
reciprocal_rank(9): 0
avg_precision(10): 0.3333333333333333
recall(10): 0.08333333333333333
reciprocal_rank(10): 0.3333333333333333
avg_precision(11): 1.0
recall(11): 0.25
reciprocal_rank(11): 1.0
avg_precision(12): 0
